In [5]:
#import excel
import pandas as pd
import numpy as np
from pandas import DataFrame

df=pd.read_excel('station.xlsx')
df.describe()

,Unnamed: 0,สถานี,Unnamed: 2,E,N,เปิด,ปิด,Unnamed: 13,Unnamed: 14,Unnamed: 15,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
count,148.000000,37.000000,37.000000,37.000000,3.700000e+01,37.000000,37.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,12616.810811,12616.540541,81184.135135,487276.225297,2.071517e+06,2510.810811,2560.297297,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,26649.007360,26925.124856,31376.389814,29538.585809,5.335286e+04,22.485598,0.701778,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,7013.000000,7013.000000,70013.000000,432179.000000,1.952092e+06,2463.000000,2558.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,7162.000000,7162.000000,70162.000000,466466.000000,2.045273e+06,2495.000000,2560.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,7480.000000,7480.000000,70480.000000,494508.000000,2.079270e+06,2507.000000,2560.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,7751.000000,7751.000000,70751.000000,512746.000000,2.108047e+06,2531.000000,2561.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,171181.000000,171181.000000,170181.000000,535859.000000,2.170800e+06,2547.000000,2561.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Convert WGS84 to Lon-Lat
import pyproj

from functools import partial
from pyproj import Proj, transform

proj_wgs84 = Proj(init="epsg:24047")
proj_gk4 = Proj(init="epsg:4326")

transformer = partial(transform, proj_wgs84, proj_gk4)
lon, lat = transformer(df['E'].values, df['N'].values)

station=df['สถานี']

columns = ['station','lon','lat']

data = {'station': station, 'Lontitude': lon, 'Latitude': lat}

#save new data
#Prog format
#https://www.spatialreference.org/ref/epsg/24047/

data = pd.DataFrame(data)

#clean
new= data.dropna()
new.to_csv('new.csv')
new.to_excel('new.xlsx')
new.describe()

C:\Users\ACER\anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\ACER\anaconda3\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-6-c22ab0dac834>:11: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transformer(df['E'].values, df[

,station,Lontitude,Latitude
count,37.000000,37.000000,37.000000
mean,12616.540541,98.874783,18.738035
std,26925.124856,0.279755,0.482369
min,7013.000000,98.355212,17.658734
25%,7162.000000,98.678500,18.500009
50%,7480.000000,98.942867,18.808329
75%,7751.000000,99.116136,19.068316
max,171181.000000,99.335876,19.635171


In [7]:
#if convert WGS84 to Lon-Lat to wgs84

from pyproj import Proj
p = pyproj.Proj(proj='utm',zone=47,ellps='WGS84', preserve_units=False)

xx, yy = p(lon,lat)
print(xx,yy)

[496879.17234184 512599.20108562 504574.1710106  ...             inf
             inf             inf] [2083277.82168916 2072709.86047413 2084138.8411035  ...              inf
              inf              inf]


In [8]:
import folium

import pandas as pd

station_map = folium.Map(location=[18.5,98.5], zoom_start=8.5)
df = pd.DataFrame(pd.read_csv('new.csv'))

locate = {}

for i, j, k in zip(df['station'], df['Latitude'], df['Lontitude']):
    temp = []
    temp.extend((j,k))
    locate['loc'] = temp
    marker = folium.Marker(location=locate['loc'], popup=str(k)+' Type:'+str(j))
    marker.add_to(station_map)

station_map